<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


$ \ $

# ***Space X  Falcon 9 First Stage Landing Prediction***

$ \ $

## ***Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia***

$ \ $

In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled as:


$$\color{aquamarine}{\text{List of Falcon 9 and Falcon Heavy launches.}}$$



[https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches](https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01)

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/Falcon9\_rocket_family.svg)


$ \ $

The launch records are stored in a HTML table shown below:

$ \ $

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/falcon9-launches-wiki.png)

$ \ $ 

-----

## ***Objectives.***

Web scrap (raspado web) `Falcon 9` launch records with BeautifulSoup:

*   Extract a Falcon 9 launch records HTML table from Wikipedia

*   Parse the table and convert it into a Pandas data frame

$ \ $ 

-----


## ***Install packages.***

$(1)$ First let's import required packages for this lab

In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

$ \ $

$(2)$ We provide some helper functions for you to process web scraped HTML table.

In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]


In [4]:
def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i %2 ==0][0:-1])
    return out


In [5]:
def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out
 

In [6]:
def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


In [7]:
def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name   

$ \ $

$(3)$ To keep the lab tasks consistent, you will be asked to scrape (raspar) the data from a snapshot (instantánea) of the 

$$\color{aquamarine}{\text{List of Falcon 9 and Falcon Heavy launches}}$$

Wikipage updated on `9th June 2021`.

In [8]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

$ \ $

-----


## ***TASK 1: Request the Falcon9 Launch Wiki page from its URL.***


$(1)$ Let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [9]:
response = requests.get(static_url)

$ \ $

$(2)$ Create a `BeautifulSoup` object from the HTML `response`.


In [10]:
soup = BeautifulSoup(response.content, 'html.parser') 
# soup

$ \ $

$(3)$ Print the page title to verify if the `BeautifulSoup` object was created properly.


In [11]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

$ \ $

----

## ***TASK 2: Extract all column/variable names from the HTML table header.***




$(1)$ Next, we want to collect all relevant column names from the HTML table header. Let's try to find all tables on the wiki page first. 



In [12]:
html_tables = soup.find_all("table")
#html_tables

$ \ $ 

$(2)$ Starting from the third table is our target table contains the actual launch records.

In [ ]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

$ \ $ 

$(3)$ You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


$ \ $

$(4)$ Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one.


In [14]:
column_names = []
resultado = first_launch_table.find_all('th')

# Iterate each th element and apply the provided extract_column_from_header() to get a column name
for x in resultado:
    # Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
    name = extract_column_from_header(x)
    if (name != None and len(name) > 0):
        column_names.append(name)


In [15]:
column_names

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

$ \ $

-----

##***TASK 3: Create a data frame by parsing the launch HTML tables.***

$(1)$ We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe

In [16]:
launch_dict = dict.fromkeys(column_names)
launch_dict

{'Flight No.': None,
 'Date and time ( )': None,
 'Launch site': None,
 'Payload': None,
 'Payload mass': None,
 'Orbit': None,
 'Customer': None,
 'Launch outcome': None}

In [17]:
# Remove an irrelvant column
del launch_dict['Date and time ( )']

In [18]:
# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []


# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

$ \ $

$(2)$ Next, we just need to fill up the launch_dict with launch records extracted from table rows.

* Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as: 

>> Reference links `B0004.1[8]`

>> Missing values `N/A [e]`

>> Inconsistent formatting

>> Etc.

* To simplify the parsing process, we have provided an incomplete code snippet (retazo) below to help you to fill up the launch_dict. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:

In [19]:
extracted_row = 0

#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):

        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False

        #get table element 
        row = rows.find_all('td')

        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1                

            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            launch_dict["Flight No."].append(flight_number) 
            # Flight Number value (Valor del número de vuelo)
            datatimelist = date_time(row[0])
            
            # Date value
            date = datatimelist[0].strip(',')
            # TODO: Append the date into launch_dict with key `Date`
            launch_dict["Date"].append(date)
            
            
            # Time value            
            time = datatimelist[1]
            # TODO: Append the time into launch_dict with key `Time`
            launch_dict["Time"].append(time)
           
              
            # Booster version
            bv = booster_version(row[1])
            # TODO: Append the bv into launch_dict with key `Version Booster`
            launch_dict["Version Booster"].append(bv)


            if not(bv):
                bv=row[1].a.string


            # Launch Site
            launch_site = row[2].a.string
            # TODO: Append the bv into launch_dict with key `Launch site`
            launch_dict["Launch site"].append(bv)

            
            # Payload
            payload = row[3].a.string            
            # TODO: Append the payload into launch_dict with key `Payload`
            launch_dict["Payload"].append(payload)

            
            # Payload Mass            
            payload_mass = get_mass(row[4])
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            launch_dict["Payload mass"].append(payload_mass)

            
            # Orbit            
            orbit = row[5].a.string
            # TODO: Append the orbit into launch_dict with key `Orbit`
            launch_dict["Orbit"].append(orbit)

            
            # Customer            
            customer = row[6].text.strip()
            # TODO: Append the customer into launch_dict with key `Customer`
            launch_dict["Customer"].append(customer)

            
            # Launch outcome            
            launch_outcome = list(row[7].strings)[0]
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_dict["Launch outcome"].append(launch_outcome)

            
            # Booster landing            
            booster_landing = landing_status(row[8])
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            launch_dict["Booster landing"].append(booster_landing)
                      

$ \ $

$(3)$ After you have fill in the parsed launch record values into launch_dict, you can create a dataframe from it.

In [20]:
df=pd.DataFrame(launch_dict)
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,F9 v1.0B0003.1,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,F9 v1.0B0004.1,Dragon,0,LEO,".mw-parser-output .plainlist ol,.mw-parser-out...",Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,F9 v1.0B0005.1,Dragon,525 kg,LEO,NASA (COTS),Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,F9 v1.0B0006.1,SpaceX CRS-1,"4,700 kg",LEO,NASA (CRS),Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,F9 v1.0B0007.1,SpaceX CRS-2,"4,877 kg",LEO,NASA (CRS),Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10


$ \ $


$(4)$ We can now export it to a CSV for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.


In [21]:
df.to_csv('spacex_web_scraped.csv', index=False)